### bash script to calculate KE
To find the files:  

REF08 = /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/
FUT08 = /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/  
WORK = /data0/project/drakkar/USERS/jrieck/  

And CDFTOOLS are here

CDF = /home/ctalandi/TOOLS/CDFTOOLS/bin

In [ ]:
cd $REF08
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};                
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-REF08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-REF08-KE/${fileK} $WORK/CREG12.L75-REF08-KE/${fileK};            
    done; 
done

cd $FUT08
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};               
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-FUT08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-FUT08-KE/${fileK} $WORK/CREG12.L75-FUT08-KE/${fileK};     
    done; 
done

CDFTOOLS to calculate Richardson number

In [ ]:
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-REF08-RI/${fileR};              
    done; 
done

cd $WORK
for yyyy in {2051..2059}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-FUT08-RI/${fileR};              
    done; 
done

### calculate climatological seasonal cycle of U and V

need to do that first to calculate some other things. Once the other things calculated, we can compute the seasonal cycles of the rest all together

In [ ]:
cd $WORK/CREG12.L75-REF08-UV_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridU.nc U_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridV.nc V_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-RI/1996/CREG12.L75-REF08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    tmp2=$(echo ${tmp/RI/gridUclim})
    fileU=${tmp2/y1996/}
    fileV=${fileU/gridUclim/gridVclim}
    mv U_tmp${K}.nc ${fileU}
    mv V_tmp${K}.nc ${fileV}
    k=$(($k+1))
done

cd $WORK/CREG12.L75-FUT08-UV_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridU.nc U_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridV.nc V_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-RI/2051/CREG12.L75-FUT08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    tmp2=$(echo ${tmp/RI/gridUclim})
    fileU=${tmp2/y2051/}
    fileV=${fileU/gridUclim/gridVclim}
    mv U_tmp${K}.nc ${fileU}
    mv V_tmp${K}.nc ${fileV}
    k=$(($k+1))
done

### calculate EKE and MKE from anomalies to the climatlogy
(include interannual variations in MKE -> EKE = full field - (annual mean - long term mean + climatology))

first compute long term mean

In [ ]:
cd $WORK
ncra -O -4 -v vozocrtx $REF08M/????/CREG12.L75-REF08_y????.5d_gridU.nc CREG12.L75-REF08-UV_20yM/CREG12.L75-REF08_y1996-2015_gridU.nc
ncra -O -4 -v vomecrty $REF08M/????/CREG12.L75-REF08_y????.5d_gridV.nc CREG12.L75-REF08-UV_20yM/CREG12.L75-REF08_y1996-2015_gridV.nc

In [ ]:
cd $WORK
ncra -O -4 -v vozocrtx $FUT08M/2???/CREG12.L75-FUT08_y????.5d_gridU.nc CREG12.L75-FUT08-UV_20yM/CREG12.L75-FUT08_y2051-2070_gridU.nc
ncra -O -4 -v vomecrty $FUT08M/2???/CREG12.L75-FUT08_y????.5d_gridV.nc CREG12.L75-FUT08-UV_20yM/CREG12.L75-FUT08_y2051-2070_gridV.nc

now annual_mean - long term mean

In [ ]:
cd $WORK
fileUm=CREG12.L75-REF08-UV_20yM/CREG12.L75-REF08_y1996-2015_gridU.nc
fileVm=${fileUm/gridU/gridV}
for yyyy in {1996..2015}; do     
    echo ${yyyy};
    fileUa=$REF08M/${yyyy}/CREG12.L75-REF08_y${yyyy}.5d_gridU.nc
    fileVa=${fileUa/gridU/gridV}
    fileUam=CREG12.L75-REF08-UV_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridU_altMEAN.nc
    fileVam=${fileUam/gridU/gridV}
    ncrename -O -v vozocrtx,vozocrtxAnnualMEAN ${fileUa} ${fileUam}
    ncrename -O -v vomecrty,vomecrtyAnnualMEAN ${fileVa} ${fileVam}
    ncks -A -v vozocrtx -4 ${fileUm} ${fileUam} # here, vozocrtx is now the long term mean
    ncks -A -v vomecrty -4 ${fileVm} ${fileVam}
    ncap2 -O -4 -s 'vozocrtxAM=vozocrtxAnnualMEAN-vozocrtx' ${fileUam} ${fileUam}
    ncap2 -O -4 -s 'vomecrtyAM=vomecrtyAnnualMEAN-vomecrty' ${fileVam} ${fileVam}
done

In [ ]:
cd $WORK
fileUm=CREG12.L75-FUT08-UV_20yM/CREG12.L75-FUT08_y2051-2070_gridU.nc
fileVm=${fileUm/gridU/gridV}
for yyyy in {2051..2070}; do     
    echo ${yyyy};
    fileUa=$FUT08M/${yyyy}/CREG12.L75-FUT08_y${yyyy}.5d_gridU.nc
    fileVa=${fileUa/gridU/gridV}
    fileUam=CREG12.L75-FUT08-UV_annual-longtermMEAN/CREG12.L75-FUT08_y${yyyy}_gridU_altMEAN.nc
    fileVam=${fileUam/gridU/gridV}
    ncrename -O -v vozocrtx,vozocrtxAnnualMEAN ${fileUa} ${fileUam}
    ncrename -O -v vomecrty,vomecrtyAnnualMEAN ${fileVa} ${fileVam}
    ncks -A -v vozocrtx -4 ${fileUm} ${fileUam} # here, vozocrtx is now the long term mean
    ncks -A -v vomecrty -4 ${fileVm} ${fileVam}
    ncap2 -O -4 -s 'vozocrtxAM=vozocrtxAnnualMEAN-vozocrtx' ${fileUam} ${fileUam}
    ncap2 -O -4 -s 'vomecrtyAM=vomecrtyAnnualMEAN-vomecrty' ${fileVam} ${fileVam}
done

now full - (annual minus long term + climatology)

In [ ]:
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-REF08-UV_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileE=$(echo ${fileU/gridU/EKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpU.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpV.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpU.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpV.nc
        ncks -A -v vozocrtx -4 ${fileU} tmpU.nc
        ncks -A -v vomecrty -4 ${fileV} tmpV.nc
        ncap2 -O -4 -s 'vozocrtxPRIME=vozocrtx-(vozocrtxAM+vozocrtxCLIM)' tmpU.nc tmpU.nc
        ncap2 -O -4 -s 'vomecrtyPRIME=vomecrty-(vomecrtyAM+vomecrtyCLIM)' tmpV.nc tmpV.nc
        ncks -O -4 -v vozocrtxPRIME tmpU.nc tmpU.nc
        ncks -O -4 -v vomecrtyPRIME tmpV.nc tmpV.nc
        ncrename -O -v vozocrtxPRIME,vozocrtx tmpU.nc tmpU.nc
        ncrename -O -v vomecrtyPRIME,vomecrty tmpV.nc tmpV.nc
        $CDF/cdfeke -u tmpU.nc -v tmpV.nc -t ${fileT} -mke -o tmpE.nc
        ncks -O -4 -L 1 tmpE.nc tmpE.nc
        ncrename -O -v .vomke,voeke tmpE.nc CREG12.L75-REF08-EKE/${fileE}
    done; 
done

cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-REF08-UV_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileM=$(echo ${fileU/gridU/MKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpU.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpV.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpU.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpV.nc
        ncap2 -O -4 -s 'vozocrtx=vozocrtxAM+vozocrtxCLIM' tmpU.nc tmpU.nc
        ncap2 -O -4 -s 'vomecrty=vomecrtyAM+vomecrtyCLIM' tmpV.nc tmpV.nc
        $CDF/cdfeke -u tmpU.nc -v tmpV.nc -t ${fileT} -mke -o CREG12.L75-REF08-MKE/${fileM}
        ncks -A -v time_counter -4 -L 1 ${fileT} CREG12.L75-REF08-MKE/${fileM}
    done
done

In [ ]:
cd $WORK
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-FUT08-UV_annual-longtermMEAN/CREG12.L75-FUT08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileE=$(echo ${fileU/gridU/EKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpUf.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpVf.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpUf.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpVf.nc
        ncks -A -v vozocrtx -4 ${fileU} tmpUf.nc
        ncks -A -v vomecrty -4 ${fileV} tmpVf.nc
        ncap2 -O -4 -s 'vozocrtxPRIME=vozocrtx-(vozocrtxAM+vozocrtxCLIM)' tmpUf.nc tmpUf.nc
        ncap2 -O -4 -s 'vomecrtyPRIME=vomecrty-(vomecrtyAM+vomecrtyCLIM)' tmpVf.nc tmpVf.nc
        ncks -O -4 -v vozocrtxPRIME tmpUf.nc tmpUf.nc
        ncks -O -4 -v vomecrtyPRIME tmpVf.nc tmpVf.nc
        ncrename -O -v vozocrtxPRIME,vozocrtx tmpUf.nc tmpUf.nc
        ncrename -O -v vomecrtyPRIME,vomecrty tmpVf.nc tmpVf.nc
        $CDF/cdfeke -u tmpUf.nc -v tmpVf.nc -t ${fileT} -mke -o tmpEf.nc
        ncks -O -4 -L 1 tmpEf.nc tmpEf.nc
        ncrename -O -v .vomke,voeke tmpEf.nc CREG12.L75-FUT08-EKE/${fileE}
    done; 
done

cd $WORK
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-FUT08-UV_annual-longtermMEAN/CREG12.L75-FUT08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileM=$(echo ${fileU/gridU/MKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpUf.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpVf.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpUf.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpVf.nc
        ncap2 -O -4 -s 'vozocrtx=vozocrtxAM+vozocrtxCLIM' tmpUf.nc tmpUf.nc
        ncap2 -O -4 -s 'vomecrty=vomecrtyAM+vomecrtyCLIM' tmpVf.nc tmpVf.nc
        $CDF/cdfeke -u tmpUf.nc -v tmpVf.nc -t ${fileT} -mke -o CREG12.L75-FUT08-MKE/${fileM}
        ncks -A -v time_counter -4 -L 1 ${fileT} CREG12.L75-FUT08-MKE/${fileM}
    done
done

Now compute the remaining climatological cycles

In [ ]:
cd $WORK/
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step};
    ncra -O -4 -v voke -d time_counter,${fivedstep},,73 CREG12.L75-REF08-KE/????/CREG12.L75-REF08_y*.5d_KE.nc CREG12.L75-REF08-KE/clim/K_tmp${step}.nc
    ncra -O -4 -v siconc,sithic -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_icemod.nc CREG12.L75-REF08-icemod_clim/I_tmp${step}.nc
    ncra -O -4 -v votemper,vosaline,rhop_sig0,mldr10_1 -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridT.nc CREG12.L75-REF08-T_clim/T_tmp${step}.nc
    ncra -O -4 -v precip,qt -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_flxT.nc CREG12.L75-REF08-FLX_clim/F_tmp${step}.nc
    ncra -O -4 -v voeke -d time_counter,${fivedstep},,73 CREG12.L75-REF08-EKE/????/CREG12.L75-REF08_y*.5d_EKE.nc CREG12.L75-REF08-EKE/clim/E_tmp${step}.nc
    ncra -O -4 -v sozotaux,utau_atmoce,utau_iceoce,uwspd10 -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridU.nc CREG12.L75-REF08-UV_clim/U_tmp${step}.nc
    ncra -O -4 -v sometauy,vtau_atmoce,vtau_iceoce,vwspd10 -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridV.nc CREG12.L75-REF08-UV_clim/V_tmp${step}.nc
    ncra -O -4 -v ssh -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridT.nc CREG12.L75-REF08-SSH_clim/S_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-RI/1996/CREG12.L75-REF08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    Ktmp=$(echo ${tmp/RI/KEclim})
    Itmp=$(echo ${tmp/RI/icemodclim})
    Ttmp=$(echo ${tmp/RI/gridTclim})
    Ftmp=$(echo ${tmp/RI/flxTclim})
    Etmp=$(echo ${tmp/RI/EKEclim})
    Utmp=$(echo ${tmp/RI/U2Dclim})
    Vtmp=$(echo ${tmp/RI/V2Dclim})
    Stmp=$(echo ${tmp/RI/SSHclim})
    fileK=${Ktmp/y1996/}
    fileI=${Itmp/y1996/}
    fileT=${Ttmp/y1996/}
    fileF=${Ftmp/y1996/}
    fileE=${Etmp/y1996/}
    fileU=${Utmp/y1996/}
    fileV=${Vtmp/y1996/}
    fileS=${Stmp/y1996/}
    mv CREG12.L75-REF08-KE/clim/K_tmp${K}.nc CREG12.L75-REF08-KE/clim/${fileK}
    mv CREG12.L75-REF08-icemod_clim/I_tmp${K}.nc CREG12.L75-REF08-icemod_clim/${fileI}
    mv CREG12.L75-REF08-T_clim/T_tmp${K}.nc CREG12.L75-REF08-T_clim/${fileT}
    mv CREG12.L75-REF08-FLX_clim/F_tmp${K}.nc CREG12.L75-REF08-FLX_clim/${fileF}
    mv CREG12.L75-REF08-EKE/clim/E_tmp${K}.nc CREG12.L75-REF08-EKE/clim/${fileE}
    mv CREG12.L75-REF08-UV_clim/U_tmp${K}.nc CREG12.L75-REF08-UV_clim/${fileU}
    mv CREG12.L75-REF08-UV_clim/V_tmp${K}.nc CREG12.L75-REF08-UV_clim/${fileV}
    mv CREG12.L75-REF08-SSH_clim/S_tmp${K}.nc CREG12.L75-REF08-SSH_clim/${fileS}
    k=$(($k+1))
done

In [ ]:
cd $WORK/
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step};
    ncra -O -4 -v voke -d time_counter,${fivedstep},,73 CREG12.L75-FUT08-KE/????/CREG12.L75-FUT08_y*.5d_KE.nc CREG12.L75-FUT08-KE/clim/K_tmp${step}.nc
    ncra -O -4 -v siconc,sithic -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_icemod.nc CREG12.L75-FUT08-icemod_clim/I_tmp${step}.nc
    ncra -O -4 -v votemper,vosaline,rhop_sig0,mldr10_1 -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridT.nc CREG12.L75-FUT08-T_clim/T_tmp${step}.nc
    ncra -O -4 -v precip,qt -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_flxT.nc CREG12.L75-FUT08-FLX_clim/F_tmp${step}.nc
    ncra -O -4 -v voeke -d time_counter,${fivedstep},,73 CREG12.L75-FUT08-EKE/????/CREG12.L75FUTF08_y*.5d_EKE.nc CREG12.L75-FUT08-EKE/clim/E_tmp${step}.nc
    ncra -O -4 -v sozotaux,utau_atmoce,utau_iceoce,uwspd10 -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridU.nc CREG12.L75-FUT08-UV_clim/U_tmp${step}.nc
    ncra -O -4 -v sometauy,vtau_atmoce,vtau_iceoce,vwspd10 -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridV.nc CREG12.L75-FUT08-UV_clim/V_tmp${step}.nc
    ncra -O -4 -v ssh -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridT.nc CREG12.L75-FUT08-SSH_clim/S_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-RI/2051/CREG12.L75-FUT08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    Ktmp=$(echo ${tmp/RI/KEclim})
    Itmp=$(echo ${tmp/RI/icemodclim})
    Ttmp=$(echo ${tmp/RI/gridTclim})
    Ftmp=$(echo ${tmp/RI/flxTclim})
    Etmp=$(echo ${tmp/RI/EKEclim})
    Utmp=$(echo ${tmp/RI/U2Dclim})
    Vtmp=$(echo ${tmp/RI/V2Dclim})
    Stmp=$(echo ${tmp/RI/SSHclim})
    fileK=${Ktmp/y2051/}
    fileI=${Itmp/y2051/}
    fileT=${Ttmp/y2051/}
    fileF=${Ftmp/y2051/}
    fileE=${Etmp/y2051/}
    fileU=${Utmp/y2051/}
    fileV=${Vtmp/y2051/}
    fileS=${Stmp/y2051/}
    mv CREG12.L75-FUT08-KE/clim/K_tmp${K}.nc CREG12.L75-FUT08-KE/clim/${fileK}
    mv CREG12.L75-FUT08-icemod_clim/I_tmp${K}.nc CREG12.L75-FUT08-icemod_clim/${fileI}
    mv CREG12.L75-FUT08-T_clim/T_tmp${K}.nc CREG12.L75-FUT08-T_clim/${fileT}
    mv CREG12.L75-FUT08-FLX_clim/F_tmp${K}.nc CREG12.L75-FUT08-FLX_clim/${fileF}
    mv CREG12.L75-FUT08-EKE/clim/E_tmp${K}.nc CREG12.L75-FUT08-EKE/clim/${fileE}
    mv CREG12.L75-FUT08-UV_clim/U_tmp${K}.nc CREG12.L75-FUT08-UV_clim/${fileU}
    mv CREG12.L75-FUT08-UV_clim/V_tmp${K}.nc CREG12.L75-FUT08-UV_clim/${fileV}
    mv CREG12.L75-FUT08-SSH_clim/S_tmp${K}.nc CREG12.L75-FUT08-SSH_clim/${fileS}
    k=$(($k+1))
done

#### Calculate N$^{2}$

In [ ]:
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileT in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridT.nc; do        
        fileN=$(echo ${fileT/gridT/Nsquared} | rev | cut -d'/' -f-2 | rev);                 
        /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-REF08-Nsquared/${fileN};              
    done; 
done
for fileT in CREG12.L75-REF08-T_clim/CREG12.L75-REF08_m* ; do
    fileN=$(echo ${fileT/gridTclim/Nsquared} | rev | cut -d'/' -f-1 | rev);                 
    /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-REF08-Nsquared/clim/${fileN}
done

In [ ]:
cd $WORK
for yyyy in {2051..2070}; do  
    echo ${yyyy}
    for fileT in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridT.nc; do        
        fileN=$(echo ${fileT/gridT/Nsquared} | rev | cut -d'/' -f-2 | rev);                 
        /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-FUT08-Nsquared/${fileN};              
    done;  
done
for fileT in CREG12.L75-FUT08-T_clim/CREG12.L75-FUT08_m* ; do
    fileN=$(echo ${fileT/gridTclim/Nsquared} | rev | cut -d'/' -f-1 | rev);                 
    /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-FUT08-Nsquared/clim/${fileN}
done

#### Calculate vertical shear of velocity at this point
It is calculated in python: [calc_vertical_chear.ipynb](calc_vertical_shear.ipynb)

### bash scripts to extract points
Using only ncks seems to work just fine. This way I can extract several variables at once. This takes a while but if I start extraction of each mooring point in parallel, it can be done overnight for one simulation.

In [ ]:
I=540
J=1530
number=001

I=820
J=1250
number=002

I=960
J=1150
number=003

I=650
J=1220
number=004

I=1040
J=1320
number=005

I=840
J=1540
number=006

I=870
J=1660
number=007

I=1200
J=1000
number=008

I=660
J=1550
number=009

I=540
J=1620
number=WAL

I=1080
J=1460
number=WLS

I=1070
J=1400
number=WSZ

In [ ]:
I=540
J=1530
number=001
for year in {1996..2015}; do
    echo $year
    cd $REF08
    echo "gridT"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v votemper,vosaline,rhop_sig0,mldr10_1 $file $WORK/CREG12.L75-REF08-moorings/$file.votemper.profile.I0${I}J${J}; done
    echo "icemod"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v siconc,sithic $file $WORK/CREG12.L75-REF08-moorings/$file.icemod.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-RI
    echo "voric"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_RI.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voric $file $WORK/CREG12.L75-REF08-moorings/$file.voric.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-KE
    echo "voke"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_KE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voke $file $WORK/CREG12.L75-REF08-moorings/$file.voke.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-EKE
    echo "voeke"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_EKE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voeke $file $WORK/CREG12.L75-REF08-moorings/$file.voeke.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-MKE
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_MKE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v vomke $file $WORK/CREG12.L75-REF08-moorings/$file.vomke.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-Nsquared
    echo "vobn2"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_Nsquared.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v vobn2 $file $WORK/CREG12.L75-REF08-moorings/$file.vobn2.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-shear
    echo "vovshear" 
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_shear.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v vovshear $file $WORK/CREG12.L75-REF08-moorings/$file.vovshear.profile.I0${I}J${J}; done
done

cd $WORK/CREG12.L75-REF08-moorings/
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 [1-2]???/*.nc.voke.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voke.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 [1-2]???/*gridT.nc.votemper.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 [1-2]???/*icemod.nc.icemod.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_icemod.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 [1-2]???/*.nc.voric.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voric.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 [1-2]???/*.nc.voeke.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voeke.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 [1-2]???/*.nc.vomke.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_vomke.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 [1-2]???/*.nc.vobn2.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_vobn2.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 [1-2]???/*.nc.vovshear.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_vovshear.nc

In [ ]:
I=540
J=1530
number=001
for year in {2051..2070}; do
    echo $year
    cd $FUT08
    echo "gridT"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_gridT.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v votemper,vosaline,rhop_sig0,mldr10_1 $file $WORK/CREG12.L75-FUT08-moorings/$file.votemper.profile.I0${I}J${J}; done
    echo "icemod"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_icemod.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v siconc,sithic $file $WORK/CREG12.L75-FUT08-moorings/$file.icemod.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-RI
    echo "voric"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_RI.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voric $file $WORK/CREG12.L75-FUT08-moorings/$file.voric.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-KE
    echo "voke"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_KE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voke $file $WORK/CREG12.L75-FUT08-moorings/$file.voke.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-EKE
    echo "voeke"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_EKE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voeke $file $WORK/CREG12.L75-FUT08-moorings/$file.voeke.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-MKE
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_MKE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v vomke $file $WORK/CREG12.L75-FUT08-moorings/$file.vomke.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-Nsquared
    echo "vobn2"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_Nsquared.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v vobn2 $file $WORK/CREG12.L75-FUT08-moorings/$file.vobn2.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-shear
    echo "vovshear" 
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_shear.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v vovshear $file $WORK/CREG12.L75-FUT08-moorings/$file.vovshear.profile.I0${I}J${J}; done
done

cd $WORK/CREG12.L75-FUT08-moorings/
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 2???/*.nc.voke.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voke.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 2???/*gridT.nc.votemper.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 2???/*icemod.nc.icemod.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_icemod.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 2???/*.nc.voric.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voric.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 2???/*.nc.voeke.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voeke.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 2???/*.nc.vomke.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_vomke.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 2???/*.nc.vobn2.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_vobn2.nc
ncrcat -O -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 2???/*.nc.vovshear.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_vovshear.nc

### End of preparations
---

In [ ]:
for file in CREG12.L75-FUT08_m??d??.5d_EKEclim.nc; do echo ${file}; nccopy -d1 ${file} tmp.nc; mv tmp.nc ${file}; done